# CORD-19 Collect SCOPUS data

In general, this jupyter notebook is designated to collect additional data via scopus to enbroaden the CORD19 dataset: 
https://datadryad.org/stash/dataset/doi:10.5061/dryad.vmcvdncs0

First, relevant packages must be imported to the Notebook.

In [1]:
import numpy as np
import pandas as pd
import csv
import ast
import collections
import matplotlib.pyplot as plt
import Levenshtein as lev
from fuzzywuzzy import fuzz 
import datetime
import matplotlib.pyplot as plt
import re
from urllib.parse import urlparse
from collections import Counter

from elsapy.elsclient import ElsClient
from elsapy.elsdoc import FullDoc, AbsDoc
from elsapy.elssearch import ElsSearch

import time # for sleep
from pybtex.database import parse_file, BibliographyData, Entry
import json
from elsapy.elsclient import ElsClient
from elsapy.elsdoc import AbsDoc
from elsapy.elssearch import ElsSearch

Get the data and save it to a variable.

In [2]:
CORD19_CSV = pd.read_csv('../data/cord-19/CORD19_software_mentions.csv')

Check the length of the column containing doi's.

In [3]:
len(CORD19_CSV['doi'])

77448

Display the column doi to see if there are inconsistencies such as NaN's

In [4]:
doi = CORD19_CSV['doi']
doi

0                                 NaN
1          10.1016/j.regg.2021.01.002
2           10.1016/j.rec.2020.08.002
3        10.1016/j.vetmic.2006.11.026
4                   10.3390/v12080849
                     ...             
77443      10.1007/s11229-020-02869-9
77444                             NaN
77445     10.1101/2020.05.13.20100206
77446      10.1007/s42991-020-00052-8
77447     10.1101/2020.09.14.20194670
Name: doi, Length: 77448, dtype: object

Create a series with solely unique values and neglect NaN's. It is important to sort the unique values. Otherwise, the method is creating different results after each restart of the notebook. 

In [5]:
doi_counted = doi.value_counts().sort_index(ascending=True)
doi_counted

10.1001/jamainternmed.2020.1369       1
10.1001/jamanetworkopen.2020.16382    1
10.1001/jamanetworkopen.2020.17521    1
10.1001/jamanetworkopen.2020.20485    1
10.1001/jamanetworkopen.2020.24984    1
                                     ..
10.9745/ghsp-d-20-00115               1
10.9745/ghsp-d-20-00171               1
10.9745/ghsp-d-20-00218               1
10.9758/cpn.2020.18.4.607             1
10.9781/ijimai.2020.02.002            1
Name: doi, Length: 74302, dtype: int64

The following function determines the requested information from the Scopus API.

In [6]:
#Adapted from https://github.com/ElsevierDev/elsapy/blob/master/exampleProg.py
def fetch_scopus_api(client, doi):
    """obtain additional paper information from scopus by doi
    """
    doc_srch = ElsSearch("DOI("+doi+")",'scopus')
    doc_srch.execute(client, get_all = True)
    #print ("doc_srch has", len(doc_srch.results), "results.")
    #print(doc_srch.results)
    try:
        scopus_id=doc_srch.results[0]["dc:identifier"].split(":")[1]
        scp_doc = AbsDoc(scp_id = scopus_id)
        if scp_doc.read(client):
            # print ("scp_doc.title: ", scp_doc.title)
            scp_doc.write()   
        else:
            print ("Read document failed.")
        # print(scp_doc.data["affiliation"])
        return scp_doc.data
    except:
        return None

Thusly, the configuration file is set up and contains an APIkey. Further information: https://github.com/ElsevierDev/elsapy/blob/master/CONFIG.md

In [7]:
con_file = open("config.json")
config = json.load(con_file)
con_file.close()

Moreover, the client is initialized with the API-Key.

In [8]:
client = ElsClient(config['apikey'])

For demonstation purposes, the following cells shows which data is returned by the Scopus API. 

In [9]:
return_example = fetch_scopus_api(client, '10.1016/j.dsx.2020.04.012')
print(json.dumps(return_example, indent=2))

{
  "affiliation": [
    {
      "affiliation-city": "New Delhi",
      "affilname": "Jamia Hamdard",
      "affiliation-country": "India"
    },
    {
      "affiliation-city": "New Delhi",
      "affilname": "Jamia Millia Islamia",
      "affiliation-country": "India"
    },
    {
      "affiliation-city": "New Delhi",
      "affilname": "Indraprastha Apollo Hospitals",
      "affiliation-country": "India"
    }
  ],
  "coredata": {
    "srctype": "j",
    "eid": "2-s2.0-85083171050",
    "pubmed-id": "32305024",
    "prism:coverDate": "2020-07-01",
    "prism:aggregationType": "Journal",
    "prism:url": "https://api.elsevier.com/content/abstract/scopus_id/85083171050",
    "dc:creator": {
      "author": [
        {
          "ce:given-name": "Raju",
          "preferred-name": {
            "ce:given-name": "Raju",
            "ce:initials": "R.",
            "ce:surname": "Vaishya",
            "ce:indexed-name": "Vaishya R."
          },
          "@seq": "1",
          "ce:init

Based on the returned data, further analysis is conductable. Therefore, two notebooks are created to analyse data linked to: 
<ul>
  <li>affiliation</li>
  <li>coredata</li>
</ul>    

Transform the dataset to an ethically correct form.

In [10]:
def remove_unethical_entries(json_holder):
    #Checking if JSON is None
    if json_holder is None:
        return json_holder
    
     #Checking if JSON is NaN
    if json_holder == "NaN":
        return None

    #JSON starts with { else [
    string_helper = str(json_holder)
    #print(string_helper[0])
    if string_helper[0] == "{":
            if 'affiliation-city' in json_holder:
                del json_holder['affiliation-city']
            if 'affilname' in json_holder:
                 del json_holder['affilname']
            if 'dc:creator' in json_holder:
                 del json_holder['dc:creator']
            return json_holder
    elif string_helper[0] == "[":
        #print(json_holder)
        for element in json_holder: 
            if 'affiliation-city' in element.keys():
                del element['affiliation-city']
            if 'affilname' in element.keys():
                 del element['affilname']
            if 'dc:creator' in element.keys():
                 del element['dc:creator']
        return json_holder

In [11]:
def transform_to_ethical_correct_data(df):
    len_df = len(df)
    i = 0

    while i < len_df:
        print(str(i) +"/"+str(len_df) + " (Data cleaning aligned to ethical guidelines)")
        df['affiliation'][i] = remove_unethical_entries(df['affiliation'][i])
        df['coredata'][i] = remove_unethical_entries(df['coredata'][i])
        i = i + 1
    return df

Thusly, the already fetched coredata and affiliation are read and combined to a DataFrame for further processing.

In [12]:
df_current_extra_info = pd.DataFrame()
bool_show_df = False
try:
    read_affiliation = pd.read_pickle('extra_info_affiliation_CS.pkl')
    read_coredata = pd.read_pickle('extra_info_coredata_CS.pkl')
    df_current_extra_info['affiliation'] = read_affiliation
    df_current_extra_info['coredata'] = read_coredata
    bool_show_df = True
except:
    print("The DataFrame is empty")
    bool_show_df = True

Show already fetched information if it is existent.

In [13]:
if bool_show_df == True: 
    print(df_current_extra_info)
else:
    print("There is no fetched information available.")

                                             affiliation  \
0      [{'affiliation-country': 'United States'}, {'a...   
1      [{'affiliation-country': 'United States'}, {'a...   
2      [{'affiliation-country': 'United States'}, {'a...   
3      [{'affiliation-country': 'United States'}, {'a...   
4      [{'affiliation-country': 'United States'}, {'a...   
...                                                  ...   
74297           {'affiliation-country': 'United States'}   
74298  [{'affiliation-country': 'United States'}, {'a...   
74299           {'affiliation-country': 'United States'}   
74300  [{'affiliation-country': 'Turkey'}, {'affiliat...   
74301                                               None   

                                                coredata  
0      {'srctype': 'j', 'eid': '2-s2.0-85083266658', ...  
1      {'srctype': 'j', 'prism:issueIdentifier': '7',...  
2      {'srctype': 'j', 'prism:issueIdentifier': '8',...  
3      {'srctype': 'j', 'prism:issueIdentif

The length of the DataFrame containing the current information is assigned to a variable to be used for further processing. 
Therefore, the length will be used within a while loop as a starting index. 

In [14]:
len_df_current_extra_info = len(df_current_extra_info)
len_df_current_extra_info

74302

In [15]:
def contains_only_None(dic):
    """
    This functions inspects an dictionary and returns True if it solely contains None values
    """
    return len(dic) == sum(value == None for value in dic.values())    

In [16]:
def append_fetched_data_to_df(df_current_extra_info, dic):
    """
    This function appends or inserts newly fetched data to the DataFrame containing scopus data.
    """
    #df_current_extra_info -> holding the latest data, new data needs to be appended to it, 
    #df_newly_fetched_transposed -> holdy newly fetched data, needs to be inserted or fetched
    
    if contains_only_None(dic):
        placeholder_entries = pd.DataFrame(np.empty((len(dict_new_extra_info),2),dtype=object),columns=['affiliation','coredata'], index=dict_new_extra_info.keys())
        df_newly_fetched_transposed = placeholder_entries
        print(placeholder_entries)
    else:
        #Prior appending, the dictionary is transformed to a DataFrame
        df_newly_fetched = pd.DataFrame(dic)
        #For readability, the DataFrame is transposed
        df_newly_fetched_transposed = df_newly_fetched.T
        print(df_newly_fetched_transposed)
    
    #Insert newly fetched rows which were previously not successful appended
    for index, row in df_newly_fetched_transposed.iterrows():
        #insert to current extra info DataFrame because the row is existent
        if index in df_current_extra_info.index and row is not None:
            df_current_extra_info.loc[index] = row
        #append to current extra info DataFrame because the row is new     
        if index not in df_current_extra_info.index:
            df_current_extra_info = df_current_extra_info.append(row, ignore_index=True)
            
    #returning DataFrame with inserted and replaced rows. 
    return df_current_extra_info

Both Dataframes columns are stored each to an object. The series objects are stored to each to a pkl-file which is not exceeding the size of 100MB allowing GitHub uploads.

In [17]:
def store_df_columns(df):
    df = transform_to_ethical_correct_data(df)
    ser_affiliation = df['affiliation']
    ser_coredata = df['coredata']
    ser_affiliation.to_pickle('extra_info_affiliation_CS.pkl')
    ser_coredata.to_pickle('extra_info_coredata_CS.pkl')
    return ser_affiliation, ser_coredata

Subsequently, the fetched scopus data is stored within a dictionary. Besides, the print function is used to show the state of the process by displaying the latest fetched information. 

In [18]:
%%time
dict_new_extra_info = dict()
len_dois = len(doi_counted)
def trigger_fetching():
    threshold = 0 
    i = len_df_current_extra_info
    while i < len_dois: # i < len_dois
        dict_new_extra_info[i] = fetch_scopus_api(client, doi_counted.index[i])
        print("Position fetched: " + str(i) + " -> " +  doi_counted.index[i])
        i = i + 1 
        threshold = threshold + 1
        if threshold > 99:
            df_combined_extra_info = append_fetched_data_to_df(df_current_extra_info, dict_new_extra_info)
            store_df_columns(df_combined_extra_info)
            threshold = 0
            print("batch saved")
trigger_fetching()

Wall time: 999 µs


The following cell is useful when the process above is interrupted. Therefore, the dictionary containing fetched information can be narrowed down to useful entries. 

In [19]:
# def save_new_extra_info(len_df_current_extra_info, upto):
#     """
#     This function is used to separate successfull API calls from API calls which were prevented due to an invalid API-Key.
#     As a result, this function returns a range of valid entries up to the given parameter. 
#     """
#     dict_new_extra_info_saver = dict()
#     i = len_df_current_extra_info
#     while i < upto:
#         #print("Position: " + str(i) + " -> " +  doi_counted.index[i])
#         dict_new_extra_info_saver[i] = dict_new_extra_info[i]
#         i = i + 1 
#     return dict_new_extra_info_saver

The existing and newly fetched information are combined into one DataFrame. 

In [20]:
df_combined_extra_info = append_fetched_data_to_df(df_current_extra_info, dict_new_extra_info)
df_combined_extra_info

Empty DataFrame
Columns: [affiliation, coredata]
Index: []


,affiliation,coredata
0,"[{'affiliation-country': 'United States'}, {'a...","{'srctype': 'j', 'eid': '2-s2.0-85083266658', ..."
1,"[{'affiliation-country': 'United States'}, {'a...","{'srctype': 'j', 'prism:issueIdentifier': '7',..."
2,"[{'affiliation-country': 'United States'}, {'a...","{'srctype': 'j', 'prism:issueIdentifier': '8',..."
3,"[{'affiliation-country': 'United States'}, {'a...","{'srctype': 'j', 'prism:issueIdentifier': '9',..."
4,"[{'affiliation-country': 'United States'}, {'a...","{'srctype': 'j', 'prism:issueIdentifier': '11'..."
...,...,...
74297,{'affiliation-country': 'United States'},"{'srctype': 'j', 'eid': '2-s2.0-85092678139', ..."
74298,"[{'affiliation-country': 'United States'}, {'a...","{'srctype': 'j', 'eid': '2-s2.0-85087468210', ..."
74299,{'affiliation-country': 'United States'},"{'srctype': 'j', 'eid': '2-s2.0-85092677974', ..."
74300,"[{'affiliation-country': 'Turkey'}, {'affiliat...","{'srctype': 'j', 'prism:issueIdentifier': '4',..."


In [21]:
#to big for GitHub
#df_combined_extra_info.to_csv('extra_info_CS5099.csv', sep='\t')

Both Dataframes columns are stored each to an object. The series objects are stored to each to a pkl-file which is not exceeding the size of 100MB allowing GitHub uploads.

Verifying that the returned None values are due to non existent data and not to an invalid API-Key

In [24]:
def enrich_data():
    """
    This function fetches again the scopus API and solely asks for information which previously returned None. 
    """
    #Add a new column to the DataFrame containg the DOI's which are used to fetch the API
    ser_doi = pd.Series(doi_counted.index[:len_data])
    df_current_extra_info_checker = df_combined_extra_info
    df_current_extra_info_checker['doi'] = ser_doi
    
    #fetching solely none entries 
    len_df_current_extra_info_checker = len(df_current_extra_info_checker)
    dict_new_extra_info_checker = dict()
    len_df_current_extra_info_checker = 40000
    i = 30000
    while i < len_df_current_extra_info_checker: 
        if df_current_extra_info_checker['affiliation'][i] == None or df_current_extra_info_checker['coredata'][i] == None:
            dict_new_extra_info_checker[i] = fetch_scopus_api(client, ser_doi[i])
            print("Position fetched again: " + str(i) + " -> " +  ser_doi[i])
        i = i + 1
    #check if at least one of the fetched values is not None, otherwise the process is finished
    if contains_only_None(dict_new_extra_info_checker):
        print("The scopus API did not returned new information for existing None values.")
    else:
        #There is new information to insert to the existing DataFrame
        df_combined_extra_info_fetched_again  = append_fetched_data_to_df(df_current_extra_info, dict_new_extra_info_checker)
        store_df_columns(df_combined_extra_info_fetched_again)

In [25]:
len_dois = len(doi_counted)
len_dois

74302

In [26]:
len_data = len(df_combined_extra_info)
len_data 

74302

In [ ]:
if len_dois == len_data:
    enrich_data()
else:
    print("There are entries which are not yet fetched from the scopus API.")

Position fetched again: 30887 -> 10.1016/j.vph.2020.106805
Position fetched again: 30896 -> 10.1016/j.vrih.2020.06.001
Position fetched again: 30897 -> 10.1016/j.vrih.2020.06.002
Position fetched again: 30898 -> 10.1016/j.vrih.2020.07.001
Position fetched again: 30899 -> 10.1016/j.vrih.2020.07.008
Position fetched again: 30900 -> 10.1016/j.vrih.2020.07.009
Position fetched again: 30936 -> 10.1016/j.wjam.2020.03.005
Position fetched again: 30937 -> 10.1016/j.wjorl.2020.04.007
Position fetched again: 30938 -> 10.1016/j.wjorl.2020.04.008
Position fetched again: 30939 -> 10.1016/j.wjorl.2020.05.003
Position fetched again: 30940 -> 10.1016/j.wjorl.2020.05.004
Position fetched again: 30941 -> 10.1016/j.wjorl.2021.01.001
Position fetched again: 31027 -> 10.1016/j.wpi.2020.101972
Position fetched again: 31029 -> 10.1016/j.wroa.2020.100063
Position fetched again: 31057 -> 10.1016/j.xfnr.2021.01.002
Position fetched again: 31058 -> 10.1016/j.xfre.2020.07.005
Position fetched again: 31060 -> 10.1

Position fetched again: 31868 -> 10.1016/s1773-035x(19)30258-8
Position fetched again: 31869 -> 10.1016/s1773-035x(20)30313-0
Position fetched again: 31870 -> 10.1016/s1773-035x(20)30314-2
Position fetched again: 31871 -> 10.1016/s1773-035x(20)30315-4
Position fetched again: 31891 -> 10.1016/s1936-878x(20)30146-7
Position fetched again: 31892 -> 10.1016/s1936-878x(20)30301-6
Position fetched again: 31893 -> 10.1016/s1936-878x(20)30467-8
Position fetched again: 31894 -> 10.1016/s1936-878x(20)30581-7
Position fetched again: 31895 -> 10.1016/s1936-878x(20)30777-4
Position fetched again: 31908 -> 10.1016/s2155-8256(21)00002-8
Position fetched again: 31909 -> 10.1016/s2173-5085(09)70079-6
Position fetched again: 31910 -> 10.1016/s2173-5115(08)70297-5
Position fetched again: 31954 -> 10.1016/s2213-2600(20)30556-7
Position fetched again: 32039 -> 10.1016/s2414-6447(19)30062-4
Position fetched again: 32040 -> 10.1016/s2414-6447(19)30138-1
Position fetched again: 32041 -> 10.1016/s2452-302x(20)

Position fetched again: 34335 -> 10.1038/s41467-020-18056-w
